# Create predcitors with Prophet and DeepAR+

In [37]:
import boto3
from time import sleep
import subprocess
import pandas as pd
import json
import time
import pprint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

In [38]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-2


In [39]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')
forecast_query = session.client(service_name='forecastquery')

In [40]:
# 다른 노트북에서 저장된 변수들을 불러옴
%store -r

## Hyperparameters

In [41]:
forecastHorizon = 13 # 예측할 시간의 갯수 (480시간 == 20일)
NumberOfBacktestWindows = 2 # Backtest를 하는 횟수
BackTestWindowOffset = 13 # Backtest의 평가(Evaluation)의 갯수
ForecastFrequency = "W"

아래 파라미터의 상세 정보는 [여기](https://docs.aws.amazon.com/forecast/latest/dg/metrics.html) 를 보세요.
![ForecastHorizon](static/imgs/ForecastHorizon.png)
![BackTest](static/imgs/BackTest.png)

#### Predictor algorithms

In [42]:
prophet_algorithmArn = 'arn:aws:forecast:::algorithm/Prophet'
deepAR_Plus_algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

### Prophet

In [43]:
# Prophet Specifics
prophet_predictorName= project+'_prophet_algo_1' + suffix

In [44]:
# Build Prophet:
prophet_create_predictor_response=forecast.create_predictor(PredictorName=prophet_predictorName, 
                                                  AlgorithmArn=prophet_algorithmArn,
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML= False,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                                                                         "BackTestWindowOffset": BackTestWindowOffset}, 
                                                  InputDataConfig= {"DatasetGroupArn": datasetGroupArn, "SupplementaryFeatures": [ 
                                                                     { 
                                                                        "Name": "holiday",
                                                                        "Value": "US"
                                                                     }
                                                                  ]},
                                                  FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )




### DeepAR+

In [45]:
# DeepAR+ Specifics
prophet_predictorName= project+'_deeparp_algo_1' + suffix

In [46]:
# Build DeepAR+:
deeparp_create_predictor_response=forecast.create_predictor(PredictorName=prophet_predictorName, 
                                                  AlgorithmArn=deepAR_Plus_algorithmArn,
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML= False,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                                                                         "BackTestWindowOffset": BackTestWindowOffset}, 
                                                  InputDataConfig= {"DatasetGroupArn": datasetGroupArn, "SupplementaryFeatures": [ 
                                                                     { 
                                                                        "Name": "holiday",
                                                                        "Value": "US"
                                                                     }
                                                                  ]},
                                                  FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )





## Check if the predictors are active


In [47]:
# forecast.list_predictors(Filters=[
#     {"Condition": "IS",
#      "Key": "DatasetGroupArn", 
#      "Value": datasetGroupArn}
# ])

In [59]:
#prophet_predictor_arn = 'arn:aws:forecast:us-east-2:057716757052:predictor/WalmartKaggleGonsoo_prophet_algo_174655'
prophet_predictor_arn = prophet_create_predictor_response["PredictorArn"]
forecast.describe_predictor(PredictorArn= prophet_predictor_arn)['Status']

'ACTIVE'

In [60]:
deepar_predictorArn = deeparp_create_predictor_response["PredictorArn"]
forecast.describe_predictor(PredictorArn= deepar_predictorArn)['Status']

'ACTIVE'

In [61]:
%%time
# Check the Prophet status

while True:
    createPredictorStatus = forecast.describe_predictor(PredictorArn= prophet_predictor_arn)['Status']
    print(createPredictorStatus)
    if createPredictorStatus != 'ACTIVE' and createPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

ACTIVE
CPU times: user 3.06 ms, sys: 455 µs, total: 3.52 ms
Wall time: 54 ms


In [62]:
%%time
# Check the Prophet status

while True:
    createDeeparStatus = forecast.describe_predictor(PredictorArn= deepar_predictorArn)['Status']
    print(createDeeparStatus)
    if createDeeparStatus != 'ACTIVE' and createDeeparStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

ACTIVE
CPU times: user 3.09 ms, sys: 465 µs, total: 3.55 ms
Wall time: 50 ms


## Predictor evaluation

#### Prophet

In [64]:
# Prophet Metrics
# prophet_arn = prophet_create_predictor_response['PredictorArn']
prophet_metrics = forecast.get_accuracy_metrics(PredictorArn=prophet_predictor_arn)
pp = pprint.PrettyPrinter()
pp.pprint(prophet_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Prophet',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 88226.33353738453,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.04034315671935757,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.06288963912615483,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.03310726915692263,
                                                                                          'Quantile': 0.1}]}},
                                            

In [20]:
# deepar_predictorArn = deeparp_create_predictor_response["PredictorArn"]
# forecast.describe_predictor(PredictorArn= deepar_predictorArn)

#### DeepAR+

Same as Prophet and ARIMA, now you should look at the metrics from it.

In [65]:
# DeepAR+ Metrics
deepar_predictor_arn = deepar_predictorArn
# deepar_predictor_arn = deeparp_create_predictor_response['PredictorArn']
deeparp_metrics = forecast.get_accuracy_metrics(PredictorArn=deepar_predictor_arn)
pp = pprint.PrettyPrinter()
pp.pprint(deeparp_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 91869.7812273035,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.033108388994311164,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.06262575656073387,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.026708178552466544,
                                                                                          'Quantile': 0.1}]}},
                                      

In [66]:
%store prophet_predictor_arn
%store deepar_predictor_arn

Stored 'prophet_predictor_arn' (str)
Stored 'deepar_predictor_arn' (str)
